# Advanced Object-Oriented Programming: Metaclasses and Class Decorators

A `class` is a "factory for objects (instances)"

What is a "factory for classes?"

In [1]:
class MyClass1(object):
    pass

In [2]:
obj = MyClass1()
type(obj)

__main__.MyClass1

In [3]:
type(MyClass1)

type

In [4]:
%%python2
def meta(name, bases, dct):
    return type(name, bases, dct)

class MyClass2(object):
    __metaclass__ = meta


In [5]:
def meta(name, bases, dct):
    print(name, bases, dct)
    return type(name, bases, dct)

class MyClass2(object, metaclass=meta):
    a = 'foo'


MyClass2 (<class 'object'>,) {'__module__': '__main__', '__qualname__': 'MyClass2', 'a': 'foo'}


In [6]:
dct = {'a': 'foo'}
MyClass2 = meta('MyClass2', (object,), dct)

MyClass2 (<class 'object'>,) {'a': 'foo'}


In [7]:
type(MyClass2)

type

In [8]:
class WithAMeta(type):
    def __new__(meta, name, bases, dct):
        dct['a'] = 5
        return super().__new__(meta, name, bases, dct)
    
class MyClass3(object, metaclass=WithAMeta):
    pass

In [11]:
MyClass3 = WithAMeta('MyClass3', (object,), {})

In [12]:
type(MyClass3)

__main__.WithAMeta

In [13]:
MyClass3.a

5

### Metaclass use case: collecting properties ('declarative programming')

In [27]:
class ImportantField(object):
    def __init__(self, value, name=None):
        self.value = value
        self.name = name
        
class CollectorMeta(type):
    def __new__(meta, name, bases, dct):
        important = []
        for k, v in dct.items():
            if isinstance(v, ImportantField):
                if v.name is None:
                    v.name = k
                important.append(v)
        dct['_important'] = important
        return type.__new__(meta, name, bases, dct)
        
class MyBase(object, metaclass=CollectorMeta):

    def __repr__(self):
        l = ['<{}'.format(self.__class__.__name__)]
        for fld in self._important:
            l.append(' {}={}'.format(fld.name, fld.value))
        l.append('>')
        return ''.join(l)
    
class MyClass4(MyBase):
    a = ImportantField(1)
    b = ImportantField(2)
    c = ImportantField(3)

In [28]:
MyClass4()

<MyClass4 a=1 b=2 c=3>

In [29]:
print(MyClass4())

<MyClass4 a=1 b=2 c=3>


### Metaclass use case: class registries

In [37]:
class RegistryMeta(type):
    _registry = {}
    def __new__(meta, name, bases, dct):
        if 'name' not in dct:
            raise ValueError('All instances must include a "name" attribute')
        cls = type.__new__(meta, name, bases, dct)
        meta._registry[dct['name']] = cls
        return cls
    def lookup_meta(self, name):
        return self._registry[name]

class RegistryBase(object, metaclass=RegistryMeta):
    name = 'base'
    @classmethod
    def lookup(cls, name):
        return cls._registry[name]

    
class Registered1(RegistryBase):
    name = 'r1'

class Registered2(RegistryBase):
    name = 'r2'
    
class Registered3(object, metaclass=RegistryMeta):
    name = 'r3'


In [38]:
RegistryBase.lookup('r1')

__main__.Registered1

In [39]:
RegistryBase.lookup('r2')

__main__.Registered2

In [40]:
RegistryBase.lookup_meta('r1')

__main__.Registered1

In [41]:
RegistryBase.lookup_meta('r3')

__main__.Registered3

### Metaclass use case: class operators

In [47]:
class RegistryMeta(type):
    _registry = {}
    def __new__(meta, name, bases, dct):
        if 'name' not in dct:
            raise ValueError('All instances must include a "name" attribute')
        cls = type.__new__(meta, name, bases, dct)
        meta._registry[dct['name']] = cls
        return cls
    def __getitem__(cls, name):
        return cls._registry[name]

class RegistryBase(object, metaclass=RegistryMeta):
    name = 'base'
  
#     sadly, this does not work...
#     @classmethod
#     def __getitem__(cls, name):
#         return cls._registry[name]
    
class Registered1(RegistryBase):
    name = 'r1'

class Registered2(RegistryBase):
    name = 'r2'


In [48]:
RegistryBase['r1']

__main__.Registered1

In [49]:
RegistryBase['r2']

__main__.Registered2

In [53]:
class HierMeta(type):
    def __new__(meta, name, bases, dct):
        cls = type.__new__(meta, name, bases, dct)
        return cls
    def __add__(cls, other):
        return type(f'{cls.__name__} + {other.__name__}', (cls, other), {})

class HierBase(object, metaclass=HierMeta): pass

class R1(HierBase): pass
class R2(HierBase): pass

R3 = R1 + R2


In [54]:
R3

__main__.R1 + R2

In [55]:
R3.mro()

[__main__.R1 + R2, __main__.R1, __main__.R2, __main__.HierBase, object]

### Class decorators: cheap substitute for metaclasses

In [56]:
class Registry(object):
    def __init__(self):
        self._registry = {}
    def register(self, name):
        def decorator(cls):
            self._registry[name] = cls
            return cls
        return decorator
    def __getitem__(self, name):
        return self._registry[name]
    
r = Registry()

@r.register('r1')
class Registered1:
    pass

@r.register('r2')
class Registered2():
    pass

In [57]:
r['r1']

__main__.Registered1

In [58]:
r['r2']

__main__.Registered2

## Metaclass semantics

bar = Foo()

- if Foo is a class, bar is an instance
- if Foo is a metaclass, bar is a class

- `type` is a metaclass
- `object` is class
- `'foo'` is an instance (of the class `str`, which has metaclass `type`)

# Descriptors

... or "ever wonder how `@property`, `@classmethod`, and `@staticmethod` work?"

**Descriptors** are object which contain one or more of the following magic methods, and which occur in a class body:

- `__get__(self, object, type)` - called when the descriptor attribute is looked up (e.g. `getattr()`)
- `__set__(self, object, value)` - called when the descriptor attribute is set (e.g. `setattr()`)
- `__del__(self, object)` - called when the descriptor attribute is deleted (e.g. `delattr()` or `del obj.attr`)

Let's re-implement @property:

In [79]:
class myproperty():

    def __init__(self, getter, setter=None):
        self._getter = getter
        self._setter = setter
        
    def __get__(self, object, type):
        if object is None:
            return self
        return self._getter(object)
    
    def __set__(self, object, value):
        if self._setter is None:
            raise AttributeError
        self._setter(object, value)
        
    def __call__(self, x):
        print(f'Calling myproperty({x})')
        
    def setter(self, setter):
        """Descorator to add a setter"""
        self._setter = setter
        return self
        
            

In [87]:
class Foo():
    
    @myproperty
    def bar(self):
        print('Calling getter for bar')
        return 'barval'
    
    @bar.setter
    def bar(self, value):
        print('Calling setter for bar')
        
    def get_baz(self, other=None):
        print(f'Calling get_baz({other})')
        
    baz = property(get_baz)
        
foo = Foo()

In [88]:
foo.bar

Calling getter for bar


'barval'

In [89]:
foo.bar = 10

Calling setter for bar


In [90]:
foo.baz

Calling get_baz(None)


In [91]:
foo.get_baz(10)

Calling get_baz(10)


In [63]:
Foo.bar

## Descriptor types

- A **data descriptor** is a descriptor that defines both `__get__` and `__set__`
- A **non-data descriptor** is a descriptor that defines only `__get__`

> Data and non-data descriptors differ in how overrides are calculated with respect to entries in an instance’s dictionary. If an instance’s dictionary has an entry with the same name as a data descriptor, the data descriptor takes precedence. If an instance’s dictionary has an entry with the same name as a non-data descriptor, the dictionary entry takes precedence.


In [66]:
class Foo(): 
    pass

foo = Foo()
foo.a = 5

In [67]:
foo.__dict__

{'a': 5}

In [68]:
class DataDescriptor():
    
    def __get__(self, object, type):
        if object is None:
            return self
        return 'data descriptor value'
    
    def __set__(self, object, value):
        # Just make it a read-only data descriptor
        raise AttributeError

In [69]:
class NonDataDescriptor():
    
    def __get__(self, object, type):
        if object is None:
            return self
        return 'non-data descriptor value'
        

In [70]:
class MyClass():
    data = DataDescriptor()
    nondata = NonDataDescriptor()

In [71]:
obj = MyClass()
obj.__dict__.update(
    data='instance data',
    nondata='instance nondata'
)

Data descriptors have precedence over instance data:

In [72]:
obj.data

'data descriptor value'

Instance data has precendence over non-data descriptors:

In [73]:
obj.nondata

'instance nondata'

Open [Advanced OOP Lab](./advanced-oop-lab.ipynb)